<a href="https://colab.research.google.com/github/dmitryglhf/Topomatic.LookUp/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

In [ ]:
import numpy as np
import pandas as pd

#from fedot.api.main import Fedot

In [ ]:
#df = pd.read_csv('our data...')

### QR-algorithm


In [ ]:
def householder(a):
    v = a.copy()
    v[0] += np.sign(v[0]) * np.linalg.norm(a)
    H = np.eye(len(a)) - 2 * np.outer(v, v) / np.dot(v, v)  # getting Householder matrix
    return H


def get_qr(A):
    n = A.shape[0]
    Q = np.eye(n)
    R = A.copy()

    for j in range(n):
        a = R[j:, j]  # getting vector
        H = householder(a)  # getting Householder matrix

        R[j:, j:] = H @ R[j:, j:]  # change R
        Q[:, j:] = Q[:, j:] @ H.T  # change Q

    return Q, R


def eig(A, e=1e-10):
    An = A.copy() # for eig values
    eig_vectors = np.eye(A.shape[0]) # for eig vectors
    cache_eig_values = np.diag(An) # diag values
    diff = 1000

    while diff > e:
        Q, R = get_qr(An)

        # update eigen values and eigen vectors
        An = R @ Q
        eig_vectors = eig_vectors @ Q

        # update difference between eig values
        current_eig_values = np.diag(An)
        diff = np.linalg.norm(current_eig_values - cache_eig_values)
        cache_eig_values = current_eig_values

    eig_values = np.diag(An)

    # sort eig_values and eig_vectors
    sorted_key = np.argsort(eig_values)[::-1]
    eig_values, eig_vectors = eig_values[sorted_key], eig_vectors[:,sorted_key]

    # return eigen values and eigen vectors
    return eig_values, eig_vectors

### SVD

In [ ]:
def svd(A):
    # V^T matrix
    temp_V = A.T @ A
    eig_values, V = eig(temp_V)  # Собственные значения и векторы для A^T A

    # Отсортируем собственные значения и соответствующие векторы по убыванию
    # ПРОВЕРИТЬ ЭТОТ МОМЕНТ, нужно ли сортировка, ведь я сделал её в eig()
    # подробнее посмотреть как это работает
    idx = np.argsort(eig_values)[::-1]
    eig_values = eig_values[idx]
    V = V[:, idx]
    V_T = V.T

    # U matrix
    temp_U = A @ A.T
    eig_values_U, U = eig(temp_U)  # Собственные значения и векторы для A A^T

    # Sigma matrix ПРОВЕРИТЬ ЭТОТ МОМЕНТ
    sigma = np.zeros((A.shape[0], A.shape[1]))
    for i in range(min(A.shape[0], A.shape[1])):
        sigma[i, i] = np.sqrt(eig_values[i]) if eig_values[i] >= 0 else 0

    # Result
    return U, sigma, V_T

### Data Scaling before PCA??

In [ ]:
# before pca
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X = scaler.fit_transform(df)
scaled_X.mean()

In [ ]:
cov_matrix = np.cov(scaled_X, rowvar = False)

### PCA

In [ ]:
# coming soon...

### Build the Neural Network

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html#model-parameters

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### Get Device for Training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#### Define the Class

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

### Test

In [ ]:

'''
eigen values for matrix A:
l1 = -2
l2 = 2
l3 = 4

eigen vectors for matrix A:
v1 = (0, 1, -2)
v2 = (2, -1, 0)
v3 = (9, -2, 1)

original: http://mathprofi.ru/sobstvennye_znachenija_i_sobstvennye_vektory.html
'''

A = np.array([[5, 6, 3],
                      [-1, 0, 1],
                      [1, 2, -1]], dtype = float)
A=A.T@A
print(A)
eig_values, Qk, An = eig(A)

n = A.shape[0]

#for i in range(n):
  #v = Qk[:, i]  #[x[i] for x in Qk]
  #print(np.linalg.norm((A - eig_values[i] @ np.eye(n))@v))


for i in range(n):
    v = Qk[:, i]  # Извлечение i-го собственного вектора
    # Вычисление нормы выражения (A - eig_value * I) @ v
    norm_value = np.linalg.norm((A - eig_values[i] * np.eye(n)) @ v)

    print(f"Норма для собственного значения {eig_values[i]}: {norm_value}")

#for eig in eig_values:

  #for i in range(A.shape[0]):
    #v = [x[i] for x in Qk]
    #left_side = A @ v
    #right_side = eig * v
    #print(left_side - right_side)

print('Matrix:')
print(Qk)
#print('\nEig values:')
#print(eig_values)


[[27. 32. 13.]
 [32. 40. 16.]
 [13. 16. 11.]]
Норма для собственного значения 73.00744758985249: 5.336688043150941e-14
Норма для собственного значения 4.147004321831977: 9.925850413905966e-07
Норма для собственного значения 0.8455480883155814: 9.925850409005506e-07
Matrix:
[[ 0.60030706 -0.21284941  0.77092578]
 [ 0.73491404 -0.23343806 -0.6367166 ]
 [ 0.31548817  0.94878965  0.01629151]]


### Комментарии


#### Заира

Делаем так: нам нужно найти разложение X вида $U*Сигма*V_T$:
- находим собственные значения для матрицы A*A_T и собств вектора -> определим U и Сигму из этого

Cейчас (до вс) нужно научиться сортировать в сигме синг значения по убыванию и менять соответсвующие столбцы в U.
Что делать с матрицей V я подумаю и скажу в вс

#### Дима

__Сделать__
- протестировать eig() после добавления сортировки
- добить свд и посмотреть что в нем с матрицей сигма и на кой там внутри сортировка теперь нужна, если она есть в eig()
- протестировать свд на примере (мб презентация)
- что с проверкой матрицы на симметричность и нужно ли внутри метода умножать на транспонированную чтобы сделать симметричной

#### Вадим

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его

#### Алена

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его